# Préparation des variables pour les variables d'activités 

In [ ]:
library(tidyverse)
library(factoextra)

# Charger les données
df_pilot1_2022 <- readRDS("../../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/01_pilot1_2022.rds")

In [3]:
# Préparation du jeu de données pour les variables d'activités
variables_act <- c(
  "id",
  "act_Gym",
  "act_Walk",
  "act_Run",
  "act_Yoga",
  "act_Other",
  "act_None",
  "act_Fishing",
  "act_Hunting",
  "act_VisitsMuseumsGaleries",
  "act_MotorizedOutdoorActivities",
  "act_Volunteering"
)

df_act <- df_pilot1_2022 %>%
  select(all_of(variables_act))

In [ ]:
# Tables
table(df_act$act_Gym)
table(df_act$act_Walk)
table(df_act$act_Run)
table(df_act$act_Yoga)
table(df_act$act_Other)
table(df_act$act_Non)
table(df_act$act_Fishing)
table(df_act$act_Hunting)
table(df_act$act_VisitsMuseumsGaleries)
table(df_act$act_MotorizedOutdoorActivities)
table(df_act$act_Volunteering)


### Transformations

Transformer les variables avec nuance (0, 0.25, 0.5, 0.75, 1)

Comme les 1 sont peu nombreux, il semble avantageux de transformer les variables en 0, 0.5 et 1 où

- 0.25 est intégré à 0.5
- 0.75 est intégré à 1

In [5]:
df_act$act_Gym <- as.numeric(df_act$act_Gym)
df_act$act_Walk <- as.numeric(df_act$act_Walk)
df_act$act_Run <- as.numeric(df_act$act_Run)
df_act$act_Yoga <- as.numeric(df_act$act_Yoga)
df_act$act_Other <- as.numeric(df_act$act_Other)
df_act$act_None <- as.numeric(df_act$act_None)

In [6]:
df_act$act_Fishing <- ifelse(df_act$act_Fishing == 0.25, 0.5, ifelse(df_act$act_Fishing == 0.75, 1, df_act$act_Fishing))
df_act$act_Hunting <- ifelse(df_act$act_Hunting == 0.25, 0.5, ifelse(df_act$act_Hunting == 0.75, 1, df_act$act_Hunting))
df_act$act_VisitsMuseumsGaleries <- ifelse(df_act$act_VisitsMuseumsGaleries == 0.25, 0.5, ifelse(df_act$act_VisitsMuseumsGaleries == 0.75, 1, df_act$act_VisitsMuseumsGaleries))
df_act$act_MotorizedOutdoorActivities <- ifelse(df_act$act_MotorizedOutdoorActivities == 0.25, 0.5, ifelse(df_act$act_MotorizedOutdoorActivities == 0.75, 1, df_act$act_MotorizedOutdoorActivities))
df_act$act_Volunteering <- ifelse(df_act$act_Volunteering == 0.25, 0.5, ifelse(df_act$act_Volunteering == 0.75, 1, df_act$act_Volunteering))


In [ ]:
str(df_act)

In [ ]:
table(df_act$act_Fishing)
table(df_act$act_Hunting)
table(df_act$act_VisitsMuseumsGaleries)
table(df_act$act_MotorizedOutdoorActivities)
table(df_act$act_Volunteering)

In [9]:
# Sauvegarder les données préparées
saveRDS(df_act, file = "../../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/02_pilot1_2022_act.rds")

---

### Activité de Clustering

Effectuer un exercice de clustering pour mieux saisir les données et leur relation. 

Cet exercice est purement exploratoire et sert à voir si les données peuvent être utilisées pour faire du clustering.

In [ ]:
# 2. Exploration des données

# Aperçu et exploration initiale :

# Aperçu des données
head(df_act)

# Distribution des données pour chaque variable
# table(df_act[, -which(names(df_act) == "id")]) #Trop de combinaisons

# # Vérification des données manquantes
sum(is.na(df_act))

# # Statistiques descriptives
summary(df_act %>% select(-id))

In [ ]:
str(df_act)

In [ ]:
# # Détermination du nombre optimal de clusters

#df_scaled <- scale(df_act)
df_scaled <- df_act %>% select(-id)
head(df_scaled)

#table(df_act)

In [ ]:
# Utilisons la méthode de l’élbow (coude) et l’indice de silhouette.

# Méthode du coude
fviz_nbclust(df_scaled, kmeans, method = "wss") +
  ggtitle("Méthode du coude")

# Indice de silhouette
fviz_nbclust(df_scaled, kmeans, method = "silhouette") +
  ggtitle("Indice de silhouette")

In [13]:
# Application de K-Means Clustering

# Choisissez un nombre de clusters approprié (par exemple, 3).
k <- 4

# K-Means Clustering
set.seed(123)  # Pour la reproductibilité
kmeans_result <- kmeans(df_scaled, centers = k, nstart = 25)

# Ajouter les clusters au dataframe original
df_act$cluster <- as.factor(kmeans_result$cluster)


In [ ]:
# Visualisation des clusters

# head(df_act)
kmeans_result

In [ ]:
# Visualisation des clusters

# Visualisation des clusters dans l’espace des deux premières dimensions :

# Utilisation de factoextra pour représenter les clusters
fviz_cluster(kmeans_result, data = df_scaled, ellipse.type = "euclid") +
  ggtitle("Visualisation des clusters avec K-Means")

# Graphique par variables

# Variables binaires
g_var_bin <- function(df, var) {
  # Distribution des clusters par var
  g <- ggplot(df, aes(x = .data[[var]], fill = cluster)) +
    geom_bar(position = "dodge") +
    ggtitle(paste("Distribution des clusters pour", var)) +
    theme_minimal()
  return(g)
}
g_var_bin(df_act, "act_Gym")
g_var_bin(df_act, "act_Walk")
g_var_bin(df_act, "act_Run")
g_var_bin(df_act, "act_Yoga")
g_var_bin(df_act, "act_Other")
g_var_bin(df_act, "act_None")

# Variables nuancées
g_var_nuancee <- function(df, var) {
  # Distribution des clusters par var
  g <- ggplot(df, aes(x = .data[[var]], fill = cluster)) +
    geom_histogram(binwidth = 0.25, position = "dodge") +
    ggtitle(paste("Distribution des clusters pour", var)) +
    theme_minimal()
  return(g)
}
g_var_nuancee(df_act, "act_Fishing")
g_var_nuancee(df_act, "act_Hunting")
g_var_nuancee(df_act, "act_VisitsMuseumsGaleries")
g_var_nuancee(df_act, "act_MotorizedOutdoorActivities")
g_var_nuancee(df_act, "act_Volunteering")

# # Distribution des clusters par `cons_Smoke_few_times_day`
# ggplot(df_act, aes(x = cons_Smoke_few_times_day, fill = cluster)) +
#   geom_bar(position = "dodge") +
#   ggtitle("Distribution des clusters pour cons_Smoke_few_times_day") +
#   theme_minimal()